In [0]:
import math
import random
import sys
import operator
from collections import Counter

import matplotlib.pyplot as plt 

import pandas as pd
import numpy as np

In [0]:
!rm -rf histograms
%mkdir histograms
!rm -rf data
%mkdir data

In [0]:
# L = 5
# N = 6
# ITERATIONS = 10
# MUTATION_CHANCE = 0.1
# GENE_MASK = []

In [0]:
def format_bin(lst, length):
    return list(map(lambda x: format(x, f'0{length}b'), lst))

In [0]:
def get_max_num_for_bits(bits):
    result = 0
    for i in range(0, bits):
        result |= 1 << i
    return result

print(get_max_num_for_bits(1))
print(get_max_num_for_bits(2))
print(get_max_num_for_bits(3))
print(get_max_num_for_bits(4))
print(get_max_num_for_bits(5))
print(get_max_num_for_bits(6))
print(get_max_num_for_bits(667))
print(format_bin([get_max_num_for_bits(667)+1], 670))


1
3
7
15
31
63
612360413832167804618481300175204950565278972827733245154176943827040045789568780701801476101110276210469073715640490142747228062965884323130340172286597179476547016660734615078156785793174374530940927
['0010000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000']


In [0]:
def should_mutate(Px) -> bool:
    return random.random() <= Px

def mutate(pool, Px, length):
    for i in range(0, len(pool)):
        individual = pool[i]
        for bit in range(0, length):
            if should_mutate(Px):
                individual = individual ^ (1 << bit)
        pool[i] = individual
    return pool


In [0]:
def is_bit_set(num, i):
    return num & (1 << i) > 0

In [0]:
def count_bits(num):
    return bin(num).count("1")

In [0]:
def hamming_distance(n1, n2) -> int: 
    x = n1 ^ n2
    return count_bits(x)

print(hamming_distance(2, 5) == 3)
print(hamming_distance(5, 5) == 0)
print(hamming_distance(0, 31) == 5)

True
True
True


In [0]:
def wild_type(population, length) -> int:
    wild_type = 0
    for i in range(0, length):
        ones = 0
        for individ in population:
            if is_bit_set(individ, i):
                ones += 1
        mode = round(float(ones) / len(population))
        wild_type |= (mode << i)
    return wild_type

t1 = wild_type([int('11', 2), int('11', 2)], 5) # 11 -> 3
t2 = wild_type([int('110', 2), int('101', 2), int('000', 2)], 5) # 100 -> 4

print(t1, bin(t1))
print(t2, bin(t2))



3 0b11
4 0b100


In [0]:
global_stat_fields = [
                    "Попарно: Математичне сподівання",
                    "Попарно: Середнє квадратичне відхилення",
                    "Попарно: Мода",
                    "Попарно: Коефіцієнт варіації",
                    "Попарно: Мінімальне значення",
                    "Попарно: Максимальне значення",
                    "Відсоток поліморфних нейтральних генів у популяції",
                    "Дикий тип: % Поліморфних генів",
                    "Дикий тип: Кількість поліморфних генів",
                    "Дикий тип: Математичне сподівання",
                    "Дикий тип: Середнє квадратичне відхилення",
                    "Дикий тип: Мода",
                    "Дикий тип: Коефіцієнт варіації",
                    "Дикий тип: Мінімальне значення",
                    "Дикий тип: Максимальне значення",
                    "Оптимальний: Математичне сподівання",
                    "Оптимальний: Середнє квадратичне відхилення",
                    "Оптимальний: Мода",
                    "Оптимальний: Коефіцієнт варіації",
                    "Оптимальний: Мінімальне значення",
                    "Оптимальний: Максимальне значення",
                    "Модуль різниці середнього здоров’я в популяції від оптимального",
                    "Модуль різниці найкращого здоров’я в популяції від оптимального"
]

def gen_columns(length):
    columns = ["Кількість особин у популяції"]

    for i in range(0, length + 1):
        columns.append("Попарно: Відстань " + str(i))
    for i in range(0, length + 1):
        columns.append("Оптимальний: Відстань " + str(i))
    for i in range(0, length + 1):
        columns.append("Дикий тип: Відстань " + str(i))
    
    columns.extend(global_stat_fields)

    return columns

In [0]:
def get_pair_distances(population):
    grouped_distances = dict()
    total = 0
    for row in range(0, len(population)):
        for col in range(0, len(population)):
            if row == col:
                break
            distance = hamming_distance(population[row], population[col])
            grouped_distances[distance] = grouped_distances.get(distance, 0) + 1
            total += 1

    return grouped_distances, total

def get_population_stats(population, fits, length, title, iteration):
    line = dict()
    line["Кількість особин у популяції"] = len(population)
    pairs, total_pairs = get_pair_distances(population)

    for i in range(0, length + 1):
        line[f"Попарно: Відстань {i}"] = pairs.get(i, 0) / total_pairs if total_pairs > 0 else 0

    # Попарно
    mean = sum([p[0] * (p[1] / float(total_pairs)) for p in pairs.items()]) if total_pairs > 0 else 0
    line["Попарно: Математичне сподівання"] = mean

    std = math.sqrt(sum([pow(p[0] - mean, 2) * (p[1] / float(total_pairs)) for p in pairs.items()])) if total_pairs > 0 else 0
    line["Попарно: Середнє квадратичне відхилення"] = std

    mode = max(pairs.items(), key=operator.itemgetter(1))[0] if len(pairs) > 0 else 0
    line["Попарно: Мода"] = mode

    cv = std / mean if mean > 0 else std
    line["Попарно: Коефіцієнт варіації"] = cv
    display(pairs)
    min_dist = min(pairs.keys()) if len(pairs) > 0 else 0
    line["Попарно: Мінімальне значення"] = min_dist
    max_dist = max(pairs.keys()) if len(pairs) > 0 else 0
    line["Попарно: Максимальне значення"] = max_dist

    #Цільовий
    goal_dist = Counter(map(lambda x: hamming_distance(x, 0), population))
    total_goal = sum(goal_dist.values())
    for i in range(0, length + 1):
        line[f"Оптимальний: Відстань {i}"] = goal_dist.get(i, 0) / len(population)

    goal_mean = sum([p[0] * (p[1] / float(total_goal)) for p in goal_dist.items()]) if total_pairs > 0 else 0
    line["Оптимальний: Математичне сподівання"] = mean

    goal_std = math.sqrt(sum([pow(p[0] - mean, 2) * (p[1] / float(total_goal)) for p in goal_dist.items()])) if total_goal > 0 else 0
    line["Оптимальний: Середнє квадратичне відхилення"] = goal_std

    goal_mode = max(goal_dist.items(), key=operator.itemgetter(1))[0] if len(goal_dist) > 0 else 0
    line["Оптимальний: Мода"] = goal_mode

    goal_cv = goal_std / goal_mean if goal_mean > 0 else goal_std
    line["Оптимальний: Коефіцієнт варіації"] = cv
    goal_min_dist = min(goal_dist.keys()) if len(goal_dist) > 0 else 0
    line["Оптимальний: Мінімальне значення"] = goal_min_dist
    goal_max_dist = max(goal_dist.keys()) if len(goal_dist) > 0 else 0
    line["Оптимальний: Максимальне значення"] = goal_max_dist

    wild = wild_type(population, length)
    wild_dist = Counter(map(lambda x: hamming_distance(x, 0), population))
    total_wild = sum(wild_dist.values())
    for i in range(0, length + 1):
        line[f"Дикий тип: Відстань {i}"] = wild_dist.get(i, 0) / len(population)

    wild_mean = sum([p[0] * (p[1] / float(total_wild)) for p in wild_dist.items()]) if total_pairs > 0 else 0
    line["Дикий тип: Математичне сподівання"] = mean

    wild_std = math.sqrt(sum([pow(p[0] - mean, 2) * (p[1] / float(total_wild)) for p in wild_dist.items()])) if total_wild > 0 else 0
    line["Дикий тип: Середнє квадратичне відхилення"] = wild_std

    wild_mode = max(wild_dist.items(), key=operator.itemgetter(1))[0] if len(wild_dist) > 0 else 0
    line["Дикий тип: Мода"] = wild_mode

    wild_cv = wild_std / wild_mean if wild_mean > 0 else wild_std
    line["Дикий тип: Коефіцієнт варіації"] = cv
    wild_min_dist = min(wild_dist.keys()) if len(wild_dist) > 0 else 0
    line["Дикий тип: Мінімальне значення"] = wild_min_dist
    wild_max_dist = max(wild_dist.keys()) if len(wild_dist) > 0 else 0
    line["Дикий тип: Максимальне значення"] = wild_max_dist

    all_set_bits = 0
    for locus in range(0, length):
      for ind in population:
        if is_bit_set(ind, locus):
          all_set_bits |= (1 << locus)
          continue

    # print(count_bits(all_set_bits), length)
    polymorph_bits = count_bits(all_set_bits) / float(length)

    line["Відсоток поліморфних нейтральних генів у популяції"] = polymorph_bits

    wild_polymorph = count_bits(wild)

    line["Дикий тип: % Поліморфних генів"] = wild_polymorph / float(length)
    line["Дикий тип: Кількість поліморфних генів"] = wild_polymorph

    module_mean = abs((sum(fits) / len(fits)) - fitness(0, length, 1)) #fitness of 0 always = length
    module_best = abs(max(fits) - fitness(0, length, 1))

    line["Модуль різниці середнього здоров’я в популяції від оптимального"] = module_mean
    line["Модуль різниці найкращого здоров’я в популяції від оптимального"] = module_best
    ymax=1
    xmax=length

    fig=plt.figure(figsize=(10,10))
    s='''
    - i = {}
    - відсоток поліморфних генів популяції = {:0.2f}, 
    - «Дикий тип»: % поліморфних генів = {:0.2f},
    - кількість особин в популяції = {:0.2f},
    - математичне сподівання = {:0.2f}, 
    - мода = {:0.2f}, 
    - середнє квадратичне відхилення = {:0.2f}, 
    - розмах = {:0.2f} 
    - коефіцієнт варіації = {:0.2f}.
    '''.format(iteration, polymorph_bits, wild_polymorph / float(length), len(population), 
             mean, mode, std, max_dist - min_dist,  cv )
    plt.subplot(3, 1, 1)
    plt.gcf().text(0.7, 0.85, s, fontsize=8)
    plt.ylim(0, ymax)
    plt.xlim(-1, xmax)
    plt.bar(pairs.keys(), list(map(lambda x: x / total_pairs, pairs.values())), color='b')
    plt.subplot(3, 1, 2)
    plt.ylim(0, ymax)
    plt.xlim(-1, xmax)
    plt.bar(goal_dist.keys(), list(map(lambda x: x / total_goal, goal_dist.values())), color='r')
    plt.subplot(3, 1, 3)
    plt.ylim(0, ymax)
    plt.xlim(-1, xmax)
    plt.bar(wild_dist.keys(), list(map(lambda x: x / total_wild, wild_dist.values())), color='g')
    plt.savefig(f'histograms/{title}.jpg')
    plt.close(fig)

    return line

def get_global_stats(population_stats_df):
  return population_stats_df[[
                "Попарно: Математичне сподівання",
                "Попарно: Середнє квадратичне відхилення",
                "Попарно: Мода",
                "Попарно: Коефіцієнт варіації",
                "Попарно: Мінімальне значення",
                "Попарно: Максимальне значення",
                "Відсоток поліморфних нейтральних генів у популяції",
                "Дикий тип: % Поліморфних генів",
                "Дикий тип: Кількість поліморфних генів",
                "Дикий тип: Математичне сподівання",
                "Дикий тип: Середнє квадратичне відхилення",
                "Дикий тип: Мода",
                "Дикий тип: Коефіцієнт варіації",
                "Дикий тип: Мінімальне значення",
                "Дикий тип: Максимальне значення",
                "Оптимальний: Математичне сподівання",
                "Оптимальний: Середнє квадратичне відхилення",
                "Оптимальний: Мода",
                "Оптимальний: Коефіцієнт варіації",
                "Оптимальний: Мінімальне значення",
                "Оптимальний: Максимальне значення",
                "Модуль різниці середнього здоров’я в популяції від оптимального",
                "Модуль різниці найкращого здоров’я в популяції від оптимального"
              ]].mean(axis=0)


In [0]:
class Generators:
    def one(iter: int):
        if iter == 0:
            return 1
        if iter <= 10:
            return 2 * Generators.one(iter - 1)
        return Generators.one(10)
    def two(iter: int):
        if iter == 0:
            return 1
        if iter <= 35:
            prev = Generators.two(iter - 1)
            return max(int(prev * 1.25 + 0.5), prev + 1)
        return Generators.two(35)

    three_helper = dict()

    def three(iter: int):
        first_limit = 8
        second_limit = 3008
        if iter == 0:
            return 1
        if iter > second_limit:
            return Generators.three(second_limit)

        result = Generators.three_helper.get(iter, -1)
        if result != -1:
            return result

        prev = Generators.three_helper.get(iter - 1, Generators.three(iter - 1))

        if iter <= first_limit:
            result = 2 * prev
            Generators.three_helper[iter] = result
            return result

        result = max(int(prev * 1.005 + 0.5), prev + 1)
        Generators.three_helper[iter] = result
        return result
    def four(iter: int):
        if iter == 0:
            return 10
        if iter <= 30:
            prev = Generators.four(iter - 1)
            return max(int(prev * 1.25 + 0.5), prev + 1)
        return Generators.four(30)
    def five(iter: int):
        return Generators.four(iter)
    def six(iter: int):
        return Generators.four(iter)

    seven_helper = dict()

    def seven(iter: int):
        limit = 3000
        if iter == 0:
            return 200
        if iter > limit:
            return Generators.seven(limit)

        result = Generators.seven_helper.get(iter, -1)
        if result != -1:
            return result

        prev = Generators.seven_helper.get(iter - 1, Generators.seven(iter - 1))
        result = max(int(prev * 1.005 + 0.5), prev + 1)
        Generators.seven_helper[iter] = result
        return result
    def eight(iter: int):
        return Generators.seven(iter)
    def nine(iter: int):
        return Generators.seven(iter)

# Generators()

class PopulationSizeGenerator:
    def __init__(self, option: int):
            
        __generators = {
            1: lambda iter: Generators.one(iter),
            2: lambda iter: Generators.two(iter),
            3: lambda iter: Generators.three(iter),
            4: lambda iter: Generators.four(iter),
            5: lambda iter: Generators.five(iter),
            6: lambda iter: Generators.six(iter),
            7: lambda iter: Generators.seven(iter),
            8: lambda iter: Generators.eight(iter),
            9: lambda iter: Generators.nine(iter)
        }
        self.iteration = -1
        self.generate = __generators.get(option, lambda x: x);
    def get_next(self) -> int:
        self.iteration += 1
        return self.generate(self.iteration)


In [0]:
def population(iterations=20000):
    print(f'Population with {iterations} iterations')
    for i in range(1, 10):
        gen = PopulationSizeGenerator(i)
        for j in range(0, iterations):
            gen.get_next()
        print(f"Option: {i}. Population: {gen.get_next()}")

population()
population(2000)

Population with 20000 iterations
Option: 1. Population: 1024
Option: 2. Population: 5299
Option: 3. Population: 770588176
Option: 4. Population: 8280
Option: 5. Population: 8280
Option: 6. Population: 8280
Option: 7. Population: 582804478
Option: 8. Population: 582804478
Option: 9. Population: 582804478
Population with 2000 iterations
Option: 1. Population: 1024
Option: 2. Population: 5299
Option: 3. Population: 5051640
Option: 4. Population: 8280
Option: 5. Population: 8280
Option: 6. Population: 8280
Option: 7. Population: 3976139
Option: 8. Population: 3976139
Option: 9. Population: 3976139


In [0]:
#Refactor to be generic
# def simulation():
#     pool = initial_pool(N, "zeroes")
#     df = pd.DataFrame(columns=stat_columns)

#     for i in range(0, ITERATIONS):

#         print("Iteration ", i)
#         fits = []
#         for ind in pool:
#             fit = fitness(ind, L)
#             fits.append(fit)
    
#         print("Fitnesses: ", fits)
    
#         stats = get_population_stats(pool, fits, L)
#         df = df.append(stats, ignore_index=True)

#         new_pool = rws(pool, fits, N)
#         print("Selected pool:  ", format_bin(new_pool))
#         new_pool = mutate(new_pool, MUTATION_CHANCE, L)
#         print("After mutation: ", format_bin(new_pool))

#         pool = new_pool
#     return df

In [0]:
# Add neutral, pathogen, lethal consideration
# Option 1 - fitness = length; Option 2 - Calculation of neutral, pathogenic, lethal locus
GENE_MASKS = dict()
NEUTRAL_MASK = 0
PATHOGENIC_MASK = 1
LETHAL_MASK = 2

def load_gene_masks(length):
    print(f"Importing npl_{length}.csv")
    df = pd.read_csv(f"npl_{length}.csv")
    GENE_MASKS[length] = df[f"npl_{length}"].tolist()
    print(GENE_MASKS[length])

for i in (100, 200, 1000, 2000):
    load_gene_masks(i)

def count_genes(gene_mask, mask_type):
    count = 0
    for bit in gene_mask:
        if bit == mask_type:
            count += 1
    return count

# print(f'Count lethal: {count_genes(GENE_MASKS.get(100), LETHAL_MASK)}')

def fitness(individ, length, option):
    if option == 1:
        return length
    if option != 2:
        raise SyntaxError(f"No such option for fitness: {option}")
    
    if GENE_MASKS.get(length, 0) == 0:
        load_gene_masks(length)
    
    gene_mask = GENE_MASKS.get(length)
    fitness = length
    #TODO: Refactor to use binary logical operators
    for i in range(0, length):
        if is_bit_set(individ, i) and gene_mask[length - i - 1] == LETHAL_MASK:
            return 0.1
        if is_bit_set(individ, i) and gene_mask[length - i - 1] == PATHOGENIC_MASK:
            fitness -= 10

    return fitness

# print(fitness(512 , 100, 2)) #100
# print(fitness(1024 , 100, 2)) #0.1
# print(fitness(2048 , 100, 2)) #100
# print(fitness(4096 , 100, 2)) #0.1


Importing npl_100.csv
[0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 1, 0, 0, 2, 2, 1, 2, 2, 2, 2, 0, 2, 0, 2, 2, 0, 2, 0, 2, 2, 2, 0, 2, 2, 2, 2, 2, 0, 2, 2, 0, 2, 2, 2, 0, 0, 0, 2, 2, 0, 2, 0, 2, 0, 2, 2, 2, 0, 0, 0, 2, 2, 0, 0, 2, 2, 2, 2, 0, 0, 2, 0, 0, 2, 2, 0, 0, 2, 2, 0, 2, 2, 0, 0, 2, 0, 2, 0, 2, 0, 0, 0, 2, 2, 2, 2, 0, 2, 2]
Importing npl_200.csv
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 0, 1, 2, 2, 0, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 0, 2, 0, 0, 0, 2, 2, 0, 2, 2, 2, 2, 2, 0, 1, 0, 2, 2, 0, 2, 0, 2, 2, 0, 2, 0, 0, 0, 0, 0, 2, 2, 2, 0, 2, 0, 2, 0, 0, 2, 0, 2, 2, 0, 2, 2, 2, 1, 2, 2, 2, 0, 2, 2, 2, 0, 1, 2, 0, 0, 0, 2, 1, 0, 2, 2, 2, 0, 2, 2, 2, 0, 2, 2, 0, 0, 0, 2, 0, 2, 0, 2, 2, 2, 0, 0, 0, 2, 2, 0, 2, 0, 2, 2, 2, 2, 2, 2, 0, 0, 2, 0, 2, 2, 2, 2, 2, 2, 2, 0, 0, 2, 2, 0, 2, 2, 2, 2, 0, 2, 0, 2, 2, 0, 0, 0, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 0, 0, 2, 2, 0, 2, 2, 0, 2, 0, 2, 0]
Importing npl_1000.csv
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [0]:
def calc_fits(pool, length, option):
    fits = []
    for ind in pool:
        fit = fitness(ind, length, option)
        fits.append(fit)
    return fits

In [0]:
def select_non_lethal(pool, fit):
    new_pool = []
    new_fit = []
    for i in range(0, len(pool)):
        if fit[i] <= 0.1:
            continue
        new_pool.append(pool[i])
        new_fit.append(fit[i])

    return new_pool, new_fit

In [0]:
def init_unit_normal(mu, sigma, length, fitness_option):
    hamming = int(abs(np.random.normal(mu, sigma)) + 0.5)
    # print(f'Hamming: {hamming}')
    if fitness_option == 1:
        gene_mask = []
        for i in range(0, length):
            gene_mask.append(0)
    else:
        gene_mask = GENE_MASKS.get(length)
    # print(f'Mask: {gene_mask}')
    non_lethal_genes_count = len(gene_mask) - count_genes(gene_mask, LETHAL_MASK)
    # print(f'Non lethal count: {non_lethal_genes_count}')
    mutation_bits = np.random.choice(non_lethal_genes_count, hamming, replace=False)
    # print(f'Mutation bits: {mutation_bits}')
    unit = 0
    for i in mutation_bits:
        bit_to_mutate = -1
        bit_counter = 0
        while bit_counter <= i:
            bit_to_mutate += 1
            if gene_mask[bit_to_mutate] != LETHAL_MASK:
                bit_counter +=1
        # print(f'For bit {i}, skipped {bit_to_mutate - i} lethal bits. Setting bit: {bit_to_mutate}')
        unit += 1 << (length - bit_to_mutate - 1)
    if hamming_distance(0, unit) != hamming:
        raise SyntaxError(f"Error in init_unit_normal. Wrong hamming distance. mu: {mu}, sigma: {sigma}, length: {length}, fitness_option: {fitness_option}")
    if fitness(unit, length, fitness_option) <= 0.1:
        # print(f"Error in init_unit_normal. Lethal mutation. mu: {mu}, sigma: {sigma}, length: {length}, fitness_option: {fitness_option}")
        raise SyntaxError(f"Error in init_unit_normal. Lethal mutation. mu: {mu}, sigma: {sigma}, length: {length}, fitness_option: {fitness_option}")
    return unit

# for i in range(0, 10):
#     length = 100
#     option = 2
#     unit = init_unit_normal(0, 1, length, option)
#     print(f'Unit: {format_bin([unit], length)}, Fitness: {fitness(unit, length, option)}, Unit (dec): {unit}')

def init_unit(length: int, distribution_option: str, fitness_option: int) -> int:
    options = {
        "zero": lambda ln, opt: 0,
        # How to make correct normal distribution ???
        "normal_st": lambda ln, opt: init_unit_normal(0, 1, ln, opt),
        "normal_0_3": lambda ln, opt: init_unit_normal(0, 3, ln, opt),
        "normal_0.76_1": lambda ln, opt: init_unit_normal(0.76, 1, ln, opt),
    }
    print(f"Length: {length}, Distribution option: {distribution_option}")
    return options.get(distribution_option)(length, fitness_option)

def init_pool(length: int, size: int, distribution_option: str, fitness_option: int):
    print("Initialing pool")
    pool = []
    for i in range(0, size):
        print(f"Generationg {i + 1} of {size}")
        selected = False
        while not selected:
            unit = init_unit(length, distribution_option, fitness_option)
            print(f"Trying {format_bin([unit], length)}")
            if fitness(unit, length, fitness_option) > 0.1:
                selected = True
                print(f"Appending {format_bin([unit], length)}")
                pool.append(unit)
    return pool

In [0]:
def get_percentile(probs, throw):
    i = 0
    while throw >= probs[i]:
        throw -= probs[i]
        i += 1
    return i

def rws(pool, fits, size):
    total_fit = sum(fits)

    probs = []
    for fit in fits:
        prob = float(fit) / total_fit
        probs.append(prob)
#!!!    print("Probabilities: ", probs)

    new_pool = []
    for i in range(0, size):
        throw = random.random()
        perc = get_percentile(probs, throw)
        new_pool.append(pool[perc])

    return new_pool

In [0]:
def tournament_round(pool, fit, count):
    participants = dict()

    if len(pool) - count <= 0:
        for i in range(0, len(pool)):
            participants[i] = 1
    else:
        for i in range(0, count):
            selected = False
            while not selected:
                ind = np.random.randint(len(pool))

                if participants.get(ind, -1) == -1:
                    participants[ind] = 1
                    selected = True
    
    winner = -1
    winner_fit = -1
    for key in participants:
        if fit[key] > winner_fit:
            winner = pool[key]
            winner_fit = fit[key]

    return winner

#tournament_round([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20], [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20], 12)
#print(tournament_round([1], [1], 12))

def tournament(pool, fit, size, count):
    new_pool = []
    for i in range(0, size):
        new_pool.append(tournament_round(pool, fit, count))

    return new_pool

In [0]:
def select_predecessor_pool(pool, fit, selection_option, size):
    selection_methods = {
        "rws": rws,
        "tournament_2": lambda pool, fit, size: tournament(pool, fit, size, 2),
        "tournament_12": lambda pool, fit, size: tournament(pool, fit, size, 12)}
    return selection_methods.get(selection_option)(pool, fit, size)


In [0]:
def do_iteration(i, pool, size_gen, length, fitness_option, selection_option, Px, df, title):
    # print("Pool:", format_bin(pool, length))
    fits = calc_fits(pool, length, fitness_option)
    # print("Fitnesses:", fits)

    non_lethal_pool, non_lethal_fit = select_non_lethal(pool, fits)
    if len(non_lethal_pool) == 0:
        print("Population DIED from mutations")
        return non_lethal_pool, df
    # print("Non_lethal_pool:", format_bin(non_lethal_pool, length))
    # print("Not_lethal_fitnesses:", non_lethal_fit)
    # print("Removed lethal:", len(pool) - len(non_lethal_pool))

    stats = get_population_stats(pool, fits, length, title, i)
    df = df.append(stats, ignore_index=True)

    successor_pool_size = size_gen.get_next()
    # print("Successor pool size:", successor_pool_size)
    predecessor_pool = select_predecessor_pool(non_lethal_pool, non_lethal_fit, selection_option, successor_pool_size)
    # print("Selected  pool:", format_bin(predecessor_pool, length))
    successor_pool = mutate(predecessor_pool, Px, length)
    # print("After mutation:", format_bin(successor_pool, length))
    return successor_pool, df

In [0]:
Px_values = {"rws_10_1000": 0.00004894956, "rws_10_5000": 5.84103135890646E-06, "rws_10_10000": 2.55884808484052E-06,
             "rws_20_1000": 0.00002135745, "rws_20_5000": 2.85216033312367E-06, "rws_20_10000": 1.24947882619106E-06,
             "rws_100_1000": 2.69339092197189E-06, "rws_100_5000": 4.33128819879781E-07, "rws_100_10000": 1.89745745766054E-07,
             "rws_200_1000": 1.34757278701916E-06, "rws_200_5000": 2.16705494246041E-07, "rws_200_10000": 9.49346793148726E-08,
             "rws_1000_1000": 2.52494432359449E-07, "rws_1000_5000": 4.06040633099026E-08, "rws_1000_10000": 1.77878910851695E-08,
             "rws_2000_1000": 1.17473955847569E-07, "rws_2000_5000": 1.88911885934537E-08, "rws_2000_10000": 8.27588122412854E-09,
             "tournament_2_100_1000": 4.88907478530056E-06, "tournament_2_100_5000": 7.86220512880801E-07, "tournament_2_100_10000": 3.44428702746088E-07,
             "tournament_2_200_1000": 2.51090710706304E-06, "tournament_2_200_5000": 4.0378328420063E-07, "tournament_2_200_10000": 1.76890007942165E-07,
             "tournament_2_1000_1000": 4.81301061822077E-07, "tournament_2_1000_5000": 7.73988503537616E-08, "tournament_2_1000_10000": 3.39070085105072E-08,
             "tournament_2_2000_1000": 2.40825996117682E-07, "tournament_2_2000_5000": 3.87276420381108E-08, "tournament_2_2000_10000": 1.69658655416227E-08,
             "tournament_12_100_1000": 5.00242530879202E-06, "tournament_12_100_5000": 8.04448605235351E-07, "tournament_12_100_10000": 3.52414093740542E-07,
             "tournament_12_200_1000": 2.51178443309626E-06, "tournament_12_200_5000": 4.0392436850678E-07, "tournament_12_200_10000": 1.76951814374011E-07,
             "tournament_12_1000_1000": 4.93408161080452E-07, "tournament_12_1000_5000": 7.93458137786284E-08, "tournament_12_1000_10000": 3.47599372699768E-08,
             "tournament_12_2000_1000": 2.43984369837258E-07, "tournament_12_2000_5000": 3.92355455402499E-08, "tournament_12_2000_10000": 1.7188368696267E-08
             }

population_options = {1: 1000, 2: 5000, 3: 10000,
                      4: 10000, 5: 10000, 6: 10000,
                      7: 10000, 8: 10000, 9: 10000}

# Option: 1. 1000: Population: 1024
# Option: 2. 5000: Population: 5299
# Option: 3. 10000: Population: 770588176
# Option: 4. 10000: Population: 8280
# Option: 5. 10000: Population: 8280
# Option: 6. 10000: Population: 8280
# Option: 7. 10000: Population: 582804478
# Option: 8. 10000: Population: 582804478
# Option: 9. 10000: Population: 582804478

In [0]:
distribution_options = {1: "zero", 2: "zero", 3: "zero",
                        4: "zero",5: "normal_st", 6: "normal_0_3",
                        7: "zero", 8: "normal_st", 9: "normal_0_3"}

# Include table with Px
# Px = 0.0004870242
# Px = 0.02

def play_simulation(iterations, options, chain_lengths, fitness_options, selection_options, Pm_options):
    print("Starting simulation:", "Iterations:", iterations, "Options:", options, "Chains:", chain_lengths, "Fitness:", fitness_options, "Selection:", selection_options)
    dfs = []
    global_stats = pd.DataFrame()
    for option in options:
        print("Starting simulation for option:", option)
        for chain_length in chain_lengths:
            print("Starting simulation for chain length:", chain_length)
            for fitness_option in fitness_options:
                print("Starting simulation for fitness_option:", fitness_option)
                for selection_option in selection_options:
                    print("Starting simulation for selection_option:", selection_option)

                    Px_key = f'{selection_option}_{chain_length}_{population_options[option]}'
                    Px = Px_values[Px_key]
                    print(f"Px for option {option}, selection option {selection_option}, length {chain_length} is {Px}")
                    for pm_option in Pm_options:
                        print("Starting simulation for pm_option:", pm_option)
                        Pm = Pm_lambdas[pm_option](Px)
                        print(f'Pm is caclulated by Pm option {pm_option} and is {Pm}')

                        run_name = f"opt_{option}_len_{chain_length}_fit_{fitness_option}_select_{selection_option}_pm_{pm_option}_iterations_{iterations}"

                        df = pd.DataFrame(columns=gen_columns(chain_length))

                        size_gen = PopulationSizeGenerator(option)
                        pool = init_pool(chain_length, size_gen.get_next(), distribution_options.get(option), fitness_option)
                        for iteration in range(0, iterations):
                            iter_name="{}__{:04d}".format(run_name, iteration + 1)
                            print("O:", option, " L:", chain_length, "F:", fitness_option, "S:", selection_option, "I:", iteration + 1, "of", iterations, "Pool:", len(pool))
                            pool, df = do_iteration(iteration, pool, size_gen, chain_length, fitness_option, selection_option, Px, df, iter_name)
                            if len(pool) == 0:
                                print(f"Simulation ended with DEAD population. O: {option}, L: {chain_length}, F: {fitness_option}, S: {selection_option}, Px: {Px}, Pm: {Pm}")
                                break
                        df.to_excel(f"data/output_{run_name}.xlsx")
                        run_stats_df = get_global_stats(df)
                        run_stats_df['run_name'] = run_name
                        global_stats = global_stats.append(run_stats_df, ignore_index=True)
                        dfs.append(df)
                        print("Done with pm_option:", pm_option)

                    print("Done with selection_option:", selection_option)
                print("Done with fitness_option:", fitness_option)
            print("Done with chain length:", chain_length)
        print("Done with option:", option)

    global_stats = global_stats.set_index('run_name')
    global_stats.to_excel(f"data/output_global_stats.xlsx")

    return global_stats


In [0]:
iterations = 20000

options = range(1, 10) #TODO: option 10 has special cases and is not included here. Implement option 10
chain_lengths = [10, 20, 100, 200, 1000, 2000]
fitness_options = [1, 2]
selection_options = ["rws", "tournament_2", "tournament_12"]
Pm_options = [0, 1, 2, 3]
Pm_lambdas = [lambda x: x, lambda x: x - 0.2 * x, lambda x: x - 0.5 * x, lambda x: 5 * x]

all_cases = []

def create_all_cases():
    for option in options:
        for chain_length in chain_lengths:
            for fitness_option in fitness_options:
                #Skip fitness option 2 for chains < 100
                if chain_length < 100 and fitness_option == 2:
                    continue
                for selection_option in selection_options:
                    #Use tournament selections only for fitness option 2
                    if fitness_option == 1 and selection_option != "rws":
                        continue
                    for Pm_option in Pm_options:
                        all_cases.append((option, chain_length, fitness_option, selection_option, Pm_option))

create_all_cases()
print("Total:", len(all_cases))
index = 0
for tpl in all_cases:
    index += 1
#    print(index, tpl)


Total: 648


In [0]:

# Full simulation:
# play_simulation(options, chain_lengths, iterations)
# Use all_cases
#play_simulation(iterations, options, chain_lengths, fitness_options, selection_options, Pm_options)
play_simulation(100, [1], [100], [1], ["rws"], [0])


Starting simulation: Iterations: 100 Options: [1] Chains: [100] Fitness: [1] Selection: ['rws']
Starting simulation for option: 1
Starting simulation for chain length: 100
Starting simulation for fitness_option: 1
Starting simulation for selection_option: rws
Px for option 1, selection option rws, length 100 is 2.69339092197189e-06
Starting simulation for pm_option: 0
Pm is caclulated by Pm option 0 and is 2.69339092197189e-06
Initialing pool
Generationg 1 of 1
Length: 100, Distribution option: zero
Trying ['0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000']
Appending ['0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000']
O: 1  L: 100 F: 1 S: rws I: 1 of 100 Pool: 1


{}

O: 1  L: 100 F: 1 S: rws I: 2 of 100 Pool: 2


{0: 1}

O: 1  L: 100 F: 1 S: rws I: 3 of 100 Pool: 4


...

{0: 519691, 1: 4080, 2: 5}

O: 1  L: 100 F: 1 S: rws I: 100 of 100 Pool: 1024


{0: 520711, 1: 3063, 2: 2}

Done with pm_option: 0
Done with selection_option: rws
Done with fitness_option: 1
Done with chain length: 100
Done with option: 1


,Відсоток поліморфних нейтральних генів у популяції,Дикий тип: % Поліморфних генів,Дикий тип: Коефіцієнт варіації,Дикий тип: Кількість поліморфних генів,Дикий тип: Максимальне значення,Дикий тип: Математичне сподівання,Дикий тип: Мода,Дикий тип: Мінімальне значення,Дикий тип: Середнє квадратичне відхилення,Модуль різниці найкращого здоров’я в популяції від оптимального,Модуль різниці середнього здоров’я в популяції від оптимального,Оптимальний: Коефіцієнт варіації,Оптимальний: Максимальне значення,Оптимальний: Математичне сподівання,Оптимальний: Мода,Оптимальний: Мінімальне значення,Оптимальний: Середнє квадратичне відхилення,Попарно: Коефіцієнт варіації,Попарно: Максимальне значення,Попарно: Математичне сподівання,Попарно: Мода,Попарно: Мінімальне значення,Попарно: Середнє квадратичне відхилення
run_name,,,,,,,,,,,,,,,,,,,,,,,
opt_1_len_100_fit_1_select_rws_pm_0_iterations_100,0.0137,0.0,9.31249,0.0,0.77,0.008562,0.0,0.0,0.051961,0.0,0.0,9.31249,0.77,0.008562,0.0,0.0,0.051961,9.31249,1.23,0.008562,0.0,0.0,0.072838


In [0]:
# One time generators for neutral / pathogenic / lethal mutation locus
# 0 - neutral, 1 - pathogenic, 2 - lethal
def generate_mutation_identity(length):
    neutral_first = 0.053
    neutral_other = 0.3577
    pathogenic = 0.0232

    neutral_first_count = int(length * neutral_first + 0.5)
    neutral_other_count = int(length * neutral_other + 0.5)
    pathogenic_count = int(length * pathogenic + 0.5)
    lethal_count = length - neutral_first_count - neutral_other_count - pathogenic_count
    print("N_f:", neutral_first_count, "N_o:", neutral_other_count, "N:", neutral_first_count + neutral_other_count, "P:", pathogenic_count, "L:", lethal_count)

    initial_seq = []
    resulting_seq = []
    for i in range(0, length):
        initial_seq.append(i)
        resulting_seq.append(LETHAL_MASK)

    for i in range(0, neutral_first_count):
        ind = initial_seq[0]
        del initial_seq[0]
        resulting_seq[ind] = NEUTRAL_MASK
    
    for i in range(0, neutral_other_count):
        rnd = np.random.randint(len(initial_seq))
        ind = initial_seq[rnd]
        del initial_seq[rnd]
        resulting_seq[ind] = NEUTRAL_MASK

    for i in range(0, pathogenic_count):
        rnd = np.random.randint(len(initial_seq))
        ind = initial_seq[rnd]
        del initial_seq[rnd]
        resulting_seq[ind] = PATHOGENIC_MASK

    print("Resulting:", resulting_seq)
    print("Counts:", Counter(resulting_seq))

    df = pd.DataFrame(resulting_seq, columns=[f"npl_{length}"])
    df.to_csv(f"npl_{length}.csv", index=False)    

#generate_mutation_identity(10)
#generate_mutation_identity(200)
# generate_mutation_identity(1000)
#generate_mutation_identity(2000)
#generate_mutation_identity(5000)
#generate_mutation_identity(10000)
#generate_mutation_identity(20000)
#generate_mutation_identity(80000)


In [0]:
import shutil
shutil.make_archive('histograms', 'zip', 'histograms')

'/content/histograms.zip'

1. Яка початкова популяція в 3) - нулі?
2. Кількість ітерацій в 3) з к=1.25 - з 9 по 3000, чи з 9 по 3008?
3. Як зробити нормальний і рівномірний розподіл?
4. Як брати Рх?
5. З чого будувати гістограми?
6. Таблиця зі статистикою - одна чи багато?
7. Що з прогонами?
8. Що робити якщо всі будуть летальні
9. Як відбирати турнір при N < tournament_count